<a href="https://colab.research.google.com/github/Dicki-ibnu/CleansingData/blob/main/2318118DataCleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Menyambungkan Google Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#2. Import Library

In [ ]:
import pandas as pd
import numpy as np

#3. Baca file Excel


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/data_kotor(1).xlsx'
df = pd.read_excel(file_path)

#4. Cek Informasi Dataset (Sebelum Cleansing)

In [ ]:
print("=== Informasi Dataset Sebelum Cleansing ===")
print(df.info())
print("\nJumlah Data:", df.shape)
print("\n5 Baris Pertama:")
print(df.head())

=== Informasi Dataset Sebelum Cleansing ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 12 columns):
 #   Column                                                                Non-Null Count  Dtype         
---  ------                                                                --------------  -----         
 0                                                                         463 non-null    datetime64[ns]
 1   Nama Lengkap                                                          463 non-null    object        
 2   Usia/Umur                                                             463 non-null    object        
 3   Jenis Kelamin                                                         463 non-null    object        
 4   Pekerjaan saat ini                                                    463 non-null    object        
 5   Hobi Yang Anda Minati                                                 461 non-null    object        
 6 

#5. Hapus Kolom Tanggal (Karena tidak relevan)

In [ ]:
df_clean = df.drop(df.columns[0], axis=1)

#6. Mengurutkan Nama sesuai Abjad (A-Z)

In [ ]:
df_clean = df_clean.sort_values(by="Nama Lengkap", ascending=True)
df_clean["Nama Lengkap"] = df_clean["Nama Lengkap"].str.title()

#7. Hilangkan kata tahun, Tahun, Thn, THN, thn di kolom "Usia/Umur"

In [ ]:
df_clean["Usia/Umur"] = (
    df_clean["Usia/Umur"]
    .astype(str)
    .str.replace(r"(tahun|Tahun|Thn|THN|thn|Thun|thun|thm|tahn|th|Th|TH)", "", regex=True)
    .str.strip()
)

#8. Ganti "-" dengan "Belum Bekerja" dan kapitalisasi pekerjaan

In [ ]:
df_clean["Pekerjaan saat ini"] = (
    df_clean["Pekerjaan saat ini"]
    .replace("-", "Belum Bekerja")
    .str.title()
)

#9. Hapus kolom "Lingkungan Kerja Yang Anda Inginkan"

In [ ]:
if "Lingkungan Kerja Yang Anda Inginkan" in df_clean.columns:
    df_clean = df_clean.drop(columns=["Lingkungan Kerja Yang Anda Inginkan"])

#10. Bersihkan kolom "Gaji Yang Anda Inginkan"

In [ ]:
def clean_gaji(val):
    if pd.isna(val):
        return np.nan

    val = str(val).lower().replace(" ", "")

    # Normalisasi kata-kata umum
    val = val.replace("jt", "000000").replace("juta", "000000")
    val = val.replace("keatas", "").replace("keatasnya", "")
    val = val.replace("k", "000")

    # Ambil hanya angka
    angka = "".join([c for c in val if c.isdigit()])
    if angka == "":
        return np.nan

    angka = int(angka)

    # Hitung panjang digit
    digit = len(str(angka))

    # Normalisasi ke jutaan (6 nol)
    if digit > 7:
        # contoh 50000000 (8 digit) → jadi 5000000
        angka = int(str(angka)[:7])
    elif digit < 7:
        # contoh 500000 (6 digit) → jadi 5000000
        angka = angka * (10 ** (7 - digit))

    return angka


# Terapkan ke kolom
df_clean["Gaji Yang Anda Inginkan"] = df_clean["Gaji Yang Anda Inginkan"].apply(clean_gaji)

# Ganti nilai NaN dengan median
median_gaji = df_clean["Gaji Yang Anda Inginkan"].median()
df_clean["Gaji Yang Anda Inginkan"] = df_clean["Gaji Yang Anda Inginkan"].fillna(median_gaji)


#11.  Bersihkan kolom "Apakah Anda Lebih Suka Bekerja Di Desa Atau Di Kota"

In [ ]:
col = "Apakah Anda Lebih Suka Bekerja Di Desa Atau Di Kota"

# Hilangkan kata "Di", normalisasi ke huruf kecil
df_clean[col] = (
    df_clean[col]
    .astype(str)
    .str.replace("Di", "", case=False, regex=True)  # hapus kata "Di" atau "di"
    .str.strip()
    .str.lower()
)

# Mapping untuk string aneh
mapping = {
    "dikota": "kota",
    "desa dan kota": "kota",       # kalau campuran, kita pilih "kota"
    "suka bekerja di kota": "kota"
}
df_clean[col] = df_clean[col].replace(mapping)

# Jika ada nilai selain "kota" dan "desa", ubah jadi modus
valid_values = ["kota", "desa"]
modus = df_clean[col][df_clean[col].isin(valid_values)].mode()[0]
df_clean.loc[~df_clean[col].isin(valid_values), col] = modus

#12. Persingkat kolom keterampilan

In [ ]:
# Pilih kolom keterampilan
col_skill = "Keterampilan Apa Yang Anda Miliki Serta Yang Ingin Anda Kembangkan. "

# Fungsi untuk memotong kalimat jadi beberapa kata pertama
def shorten_text(text, n=3):
    if pd.isna(text):
        return text
    words = str(text).split()
    return " ".join(words[:n])  # ambil n kata pertama

# Terapkan fungsi ke kolom
df_clean[col_skill] = df_clean[col_skill].apply(lambda x: shorten_text(x, n=3))


#13. CEK INFORMASI DATASET SETELAH CLEANSING

In [ ]:
print("\n=== Informasi Dataset Setelah Cleansing ===")
print(df_clean.info())
print("\nJumlah Data:", df_clean.shape)
print("\n5 Baris Pertama Setelah Cleansing:")
print(df_clean.head())



=== Informasi Dataset Setelah Cleansing ===
<class 'pandas.core.frame.DataFrame'>
Index: 472 entries, 106 to 471
Data columns (total 10 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   Nama Lengkap                                                          463 non-null    object 
 1   Usia/Umur                                                             472 non-null    object 
 2   Jenis Kelamin                                                         463 non-null    object 
 3   Pekerjaan saat ini                                                    463 non-null    object 
 4   Hobi Yang Anda Minati                                                 461 non-null    object 
 5   Gaji Yang Anda Inginkan                                               472 non-null    float64
 6   Apakah Anda Lebih Suka Bekerja Di Desa Atau Di Kota     

#14. melihat hasil

In [ ]:
df_clean.head(10)


,Nama Lengkap,Usia/Umur,Jenis Kelamin,Pekerjaan saat ini,Hobi Yang Anda Minati,Gaji Yang Anda Inginkan,Apakah Anda Lebih Suka Bekerja Di Desa Atau Di Kota,Anda Lebih Tertarik Pada Bidang?,Manakah Yang Lebih Anda Tertarik,Keterampilan Apa Yang Anda Miliki Serta Yang Ingin Anda Kembangkan.
106,Adhe Rizky Fauzie,17,Perempuan,Pelajar,Olahraga basket,1500000.0,kota,Seni,Teknologi,"Bermain basket,ingin berkembang"
107,Adiba Dewi Yusifa,19,Perempuan,Wiraswasta,"Rebahan, healing",2000000.0,kota,Akademik,HUKUM,"Keterampilan berusaha, skill"
178,Adin Rafael Lyano,19,Pria,Mahasiswa,sepak bola,5000000.0,kota,Non Akademik,Teknologi,bermain sepak bolaa
43,Aqila Tri Annur Fadillah,19,Perempuan,Mahasiswa,Berenang,3500000.0,kota,Non Akademik,Teknologi,Gatau
291,Aan,28,Pria,Pengusaha,Bermain piano,3000000.0,kota,Seni,Teknologi,Bermain piano
202,Ade Rahmawati,27,Perempuan,Ahli Gizi,fotografi,1400000.0,kota,Sains,Teknologi,berfoto
230,Ade Rezky Supriadi,19,Pria,Mahasiswa,Membaca buku,9000000.0,kota,Non Akademik,Matematika,Bermain musik
271,Ade Sulastri,19,Perempuan,Mahasiswi,Berenang,8000000.0,kota,Seni,Teknik,Ingin menjadi chef
225,Adinda Septyani,19,Perempuan,Mahasiswi,Bulutangkis,6000000.0,kota,Non Akademik,Teknologi,Bulutangkis
258,Adiva Arsyila Savina,19,Perempuan,Mahasiswa,Baca novel,5000000.0,kota,Seni,Teknik,Menyanyi


#15. Data disimpan ke Excel

In [ ]:
# Simpan ke Excel
df_clean.to_excel("data_bersih.xlsx", index=False)

print("Data sudah berhasil disimpan!")


Data sudah berhasil disimpan!
